In [ ]:
sh, ch = sinh, cosh

In [ ]:
α, β = var("α β") # damage per health point of the attacker
λ, μ = var("λ μ") # supportive damage that does not depend on health of the attacker
a, b = var("a b") # health points of both teams at start of fight

In [ ]:
A(t) = (a + λ/α)*ch(t*sqrt(α*β)) - (b*sqrt(β/α)+μ/sqrt(α*β))*sh(t*sqrt(α*β)) - λ/α
B(t) = (b + μ/β)*ch(t*sqrt(α*β)) - (a*sqrt(α/β)+λ/sqrt(α*β))*sh(t*sqrt(α*β)) - μ/β

In [ ]:
A(t).expand().show()

In [ ]:
A(t).show()

In [ ]:
A(t).simplify_full().show()

In [ ]:
def parse_assignments(assignments):
    params = [(str(e.left()), e.right()) for e in assignments]
    return [value for name, value in sorted(params)]

In [ ]:
def solution(*params):
    a, b, α, β, λ, μ = parse_assignments(params)
    if μ > 0 and λ > 0:
        raise NotImplementedError("Can't solve with both μ and λ not equal to zero")
    if μ > 0:
        a, b = b, a
        α, β = β, α
        λ, μ = μ, λ
    A, B = a * sqrt(α), b * sqrt(β)
    A_full_power = (A + λ/sqrt(α))^2 - λ^2/α
    B_full_power = (B)^2
    if B_full_power <= A_full_power:
        print("population case 1")
        return a/A * sqrt((A+λ*a/A)^2 - B^2) - λ/α
    print("population case 2")
    return sqrt(B^2 - (A+λ/sqrt(α))^2 + λ^2/α) / sqrt(β)

In [ ]:
def get_death_time(*params):
    a, b, α, β, λ, μ = parse_assignments(params)
    if μ > 0 and λ > 0:
        raise NotImplementedError("Can't solve with both μ and λ not equal to zero")
    if μ > 0:
        a, b = b, a
        α, β = β, α
        λ, μ = μ, λ
    A, B = a * sqrt(α), b * sqrt(β)
    #if A + λ*a/A > B:
    D = (λ^2*a^4 + B^2*a^2*A^2 - A^4*(a + λ*a^2/A^2)^2)
    A_full_power = (A + λ/sqrt(α))^2 - λ^2/α
    B_full_power = (B)^2
    assert D == (B_full_power - A_full_power) * A^2*a^2
    if A_full_power >= B_full_power:
        print("death case 1")
        X = (A^2 + λ*a + A*B) / (A^2 + λ*a - A*B)
        x = float(sqrt(float(X)))
        return ln(x) / sqrt(α*β)
    print("death case 2")
    x = (sqrt(D) - λ*a^2) / (B*a*A - (a*A^2 + λ*a^2))
    return ln(x) / sqrt(α*β)

In [ ]:
def solution2(*params):
    a, b, α, β, λ, μ = parse_assignments(params)
    if μ > 0 and λ > 0: raise NotImplementedError("Can't solve with both μ and λ not equal to zero")
    if μ > 0: a, b, α, β, λ, μ = b, a, β, α, μ, λ
    A, B = a * sqrt(α), b * sqrt(β)
    if B^2 <= (A + λ/sqrt(α))^2 - λ^2/α: return 0
    D = (λ^2*a^4 + B^2*a^2*A^2 - A^4*(a + λ*a^2/A^2)^2)
    x = exp(get_death_time(*params) * sqrt(α*β))
    print("err(x) =", float((sqrt(D)-λ*a^2)/(a*A*(B-A-λ/sqrt(α))) - x))
    print("err(x) =", float((sqrt(D)-λ*A^2/α)/(a*A*(B-A-λ/sqrt(α))) - x))
    # CG5Z
    return (sqrt(D)*sqrt(α)/A^2)/sqrt(β)
    # 2206162251f5
    t1 = (sqrt(D) - A^2)^2 / (a*A)^2 
    t2 = a^2*A^2*(B^2-(A^+λ/sqrt(α))^2)^2 / (sqrt(D)-A^2)^2
    t3 = 2*(B^2-(A+λ/sqrt(α))^2)
    return sqrt(t1 + t2 + t3) / (2*sqrt(β))
    # 2206162251f3
    y = B*(x+1/x) - (A+λ/sqrt(α))*(x-1/x)
    return y / (2*sqrt(β))
    # 2206162251f9
    n = 2*D - 2*sqrt(D)*A^2 + A^4 - A^4*λ^2/α^2
    d = A^2/sqrt(α)*(sqrt(D) - A^2)
    return n / d / (2 * sqrt(β))
    # U59
    y = (2*D-2*sqrt(D)*A^2 + A^4 - A^4*λ^2/α^2) / (A^2/sqrt(α) * (sqrt(D) - A^2))
    return y/2/sqrt(β)
    ω = B^2 + λ^2/α - (A+λ/sqrt(α))^2
    y = sqrt(ω) + (α^2 - λ^2)/(2*α*(sqrt(ω) - sqrt(α)))
    return y / sqrt(β)
from random import randint as r

In [ ]:
n = 100
params = [a==r(1, n), α==r(1, n), λ==r(1, n), b==r(1, n), β==r(1, n), μ==0]
#params = [a == 2, α == 1, λ == 2, b == 3, β == 3, μ == 0]
print(params)
death = float(get_death_time(*params))
population = float(solution(*params))
span = (t, 0, death * 2)
sum([plot(solution(*params), span, color="purple"),
     line([(death, 0), (death, population)], color="purple"),
     plot(max(A.substitute(t==death, *params),
              B.substitute(t==death, *params)), span, color="green"),
     #plot(solution2(*params), span, color="yellow"),
     plot(A.substitute(*params), span, color="red"),
     plot(B.substitute(*params), span, color="blue")])

$a$ - количество человеков в начале битвы\
$b$ - количество орков в начале битвы\
$α$ - скорость убивания орков человеками\
$β$ - скорость убивания человеков орками\
$λ$ - количество постоянно умирающих орков от молний человеческого мага\
$μ$ - количество постоянно умирающих людей от магии орчьего шамана\
$H$ - количество живых человеков в тот момент, когда человеки победят орков

$$
    H = \frac{\sqrt{{\left(a α + 2 \, λ\right)} a - {\left(b β + 2 \, μ\right)} b
    + \frac{λ^{2}}{α}}}{\sqrt{α}} - \frac{λ}{α}
$$

Люди побеждаю орков при условии, что
$$
    {\left(a α + 2 \, λ\right)} a > {\left(b β + 2 \, μ\right)} b
$$
Иначе орки убьють всех человеков.

In [ ]:
def get_general_solution_formula():
    a, b, α, β, λ, μ = map(symbolic_expression, "abαβλμ")
    if a*(a*α + 2*λ) < b*(b*β + 2*μ): a, b, α, β, λ, μ = b, a, β, α, μ, λ
    A, B = a*(a*α + 2*λ), b*(b*β + 2*μ)
    return sqrt(A - B + (λ/sqrt(α))^2)/sqrt(α) - λ/α
e = get_general_solution_formula()
show(e)
#latex(e)

In [ ]:
def solution_general(*params):
    a, b, α, β, λ, μ = parse_assignments(params)
    if a*(a*α + 2*λ) < b*(b*β + 2*μ): a, b, α, β, λ, μ = b, a, β, α, μ, λ
    A, B = a*(a*α + 2*λ), b*(b*β + 2*μ)
    return sqrt(A - B + (λ/sqrt(α))^2)/sqrt(α) - λ/α

In [ ]:
def get_death_time_general(*params):
    a, b, α, β, λ, μ = parse_assignments(params)
    if a*(a*α + 2*λ) < b*(b*β + 2*μ): a, b, α, β, λ, μ = b, a, β, α, μ, λ
    A, B = a*(a*α + 2*λ), b*(b*β + 2*μ)
    n = sqrt(A - B + λ^2/α) - μ/sqrt(β)
    d = (a*sqrt(α) + λ/sqrt(α)) - (b*sqrt(β)  + μ/sqrt(β))
    return ln(n / d) / sqrt(α * β)

In [ ]:
n = 100
params = [a==r(1, n), α==r(1, n), λ==r(1, n), b==r(1, n), β==r(1, n), μ==r(1, n)]
#params = [a == 3, α == 2, λ == 7, b == 3, β == 6, μ == 1]
print(", ".join(map(str, params)))
population = float(solution_general(*params))
death = float(get_death_time_general(*params))
span = (t, 0, death * 2)
sum([plot(population, span, color="purple"),
     line([(death, 0), (death, population)], color="purple"),
     #plot(max(A.substitute(t==death, *params),B.substitute(t==death, *params)), span, color="green"),
     #plot(solution2(*params), span, color="yellow"),
     plot(A.substitute(*params), span, color="red"),
     plot(B.substitute(*params), span, color="blue")])

### Let's check conditions of winning
$$ a\sqrt{α} + \frac{λ}{\sqrt{α}} > b\sqrt{β} + \frac{μ}{\sqrt{β}} $$

In [ ]:
def get_winner(params):
    symbol_t = symbolic_expression("t")
    t, dt = 0, 0.0005
    while A.substitute(*params, symbol_t==t) > 0 and B.substitute(*params, symbol_t==t) > 0:
        t += dt
        dt *= 1.001
    return bool(A.substitute(*params, symbol_t==t) > 0)

In [ ]:
def winner_determinant1(params):
    a, b, α, β, λ, μ = parse_assignments(params)
    return bool(a*α^0.5 + λ/α^0.5 > b*β^0.5 + μ/β^0.5)

In [ ]:
def winner_determinant2(params):
    a, b, α, β, λ, μ = parse_assignments(params)
    return bool(a*(a*α + 2*λ) > b*(b*β + 2*μ))

In [ ]:
f = winner_determinant2
errors, correct, n = 0, 0, 1000
for _ in range(2**8):
    params = [a==r(1, n), α==r(1, n), λ==r(1, n), b==r(1, n), β==r(1, n), μ==r(1, n)]
    try:
        #L = ((a*α^0.5 + λ/α^0.5)^2 - (b*β^0.5 + μ/β^0.5)^2 + (μ/β^0.5))^0.5 - μ/β^0.5
        #L = float(L / ((a*α^0.5 + λ/α^0.5) - (b*β^0.5 + μ/β^0.5)))
        #R = λ/α^0.5 - ((b*β^0.5 + μ/β^0.5)^2 - (a*α^0.5 + λ/α^0.5)^2 + (λ/α^0.5))^0.5
        if f(params) == get_winner(params):
            correct += 1
        else:
            print("\nCounterexample: " + ", ".join(map(str, params)))
            break
    except Exception as e:
        print(e)
        errors += 1
    print(f"\rerrors: {errors:<3} correct: {correct:<3}", end="")

So the winner determinant was obtained purely experimentally:

$$ a {\left(a α + 2 \, λ\right)} > b {\left(b β + 2 \, μ\right)} $$

In [ ]:
def det_death_time_general_of_looser(*params):
    a, b, α, β, λ, μ = parse_assignments(params)
    if a*(a*α + 2*λ) > b*(b*β + 2*μ): a, b, α, β, λ, μ = b, a, β, α, μ, λ
    A, B = a*(a*α + 2*λ), b*(b*β + 2*μ)
    n = sqrt(A - B + λ^2/α) - μ/sqrt(β)
    d = (a*sqrt(α) + λ/sqrt(α)) - (b*sqrt(β)  + μ/sqrt(β))
    return ln(n / d) / sqrt(α * β)

errors, correct, n = 0, 0, 16
for _ in range(10**4):
    params = [a==r(1, n), α==r(1, n), λ==r(1, n), b==r(1, n), β==r(1, n), μ==r(1, n)]
    try:
        t_win = float(get_death_time_general(*params))
        t_los = float(det_death_time_general_of_looser(*params))
        if t_win <= t_los:
            correct += 1
        else:
            print("\nCounterexample: " + ", ".join(map(str, params)))
            print()
            break
    except Exception as e:
        errors += 1
    print(f"\rerrors: {errors:<3} correct: {correct:<3}", end="")

In [ ]:
def f(params, cond):
    a, b, α, β, λ, μ = parse_assignments(params)
    if cond: a, b, α, β, λ, μ = b, a, β, α, μ, λ
    A, B = a*(a*α + 2*λ), b*(b*β + 2*μ)
    n = sqrt(A - B + λ^2/α) - μ/sqrt(β)
    d = (a*sqrt(α) + λ/sqrt(α)) - (b*sqrt(β)  + μ/sqrt(β))
    return ln(n / d) / sqrt(α * β)

In [ ]:
params = [a==a, α==α, λ==λ, b==b, β==β, μ==μ]
solve(f(params, True) < f(params, False), a, α, λ, b, β, μ)

In [ ]:
sum([plot(diff(A, t).substitute(*params), span, color="blue"),
     plot(diff(B, t).substitute(*params), span, color="red")])

In [ ]:
A(t)

In [ ]:
Ax = (a + λ/α)*(x + 1/x)/2 - (b*sqrt(β/α) + μ/sqrt(α*β))*(x-1/x)/2 - λ/α
Bx = (b + μ/β)*(x+1/x)/2- (a*sqrt(α/β) + λ/sqrt(α*β))*(x-1/x)/2 - μ/β
solutions = solve(Ax == 0, x)
solutions

In [ ]:
bool((Ax.substitute(solutions[0]) == 0).expand())

In [ ]:
# general case 
for sol in solutions:
    sol.show()

In [ ]:
# special case with zero support 
for sol in solutions:
    sol.substitute(λ==0, μ==0).show()

In [ ]:
(symbolic_expression("B") == Bx.substitute(solutions[0]).substitute(λ==0, μ==0).expand().simplify_full()).show()
(symbolic_expression("B") == b * sqrt(1 - (α*a^2)/(β*b^2))).show()

In [ ]:
((Bx.substitute(solutions[0]).substitute(λ == 0, μ == 0).expand().simplify_full() ^ 2
    == (b * sqrt(1 - (α * a ^ 2) / (β * b ^ 2))) ^ 2)
    .expand().simplify_full().show())

In [ ]:
(Ax.substitute(solutions[0]) == Ax.substitute(solutions[0]).simplify_full()).show()